In [1]:
import gensim
import os
import collections
import smart_open
import random
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [2]:
perfect_answer =  "Data science is a multi-disciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from data in various forms, both structured and unstructured, similar to data mining. Data science is a concept to unify statistics, data analysis, machine learning and their related methods in order to understand and analyze actual phenomena with data. It employs techniques and theories drawn from many fields within the context of mathematics, statistics, information science, and computer science."
recorded_answer = "Data science is the field of study that combines domain expertise, programming skills, and knowledge of math and statistics to extract meaningful insights from data. Data science practitioners apply machine learning algorithms to numbers, text, images, video, audio, and more to produce artificial intelligence (AI) systems that perform tasks which ordinarily require human intelligence. In turn, these systems generate insights that analysts and business users translate into tangible business value."

In [3]:
train_answer = gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(perfect_answer), [0])
test_answer = gensim.utils.simple_preprocess(recorded_answer)

In [4]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)

In [5]:
train_answer

TaggedDocument(words=['data', 'science', 'is', 'multi', 'disciplinary', 'field', 'that', 'uses', 'scientific', 'methods', 'processes', 'algorithms', 'and', 'systems', 'to', 'extract', 'knowledge', 'and', 'insights', 'from', 'data', 'in', 'various', 'forms', 'both', 'structured', 'and', 'unstructured', 'similar', 'to', 'data', 'mining', 'data', 'science', 'is', 'concept', 'to', 'unify', 'statistics', 'data', 'analysis', 'machine', 'learning', 'and', 'their', 'related', 'methods', 'in', 'order', 'to', 'understand', 'and', 'analyze', 'actual', 'phenomena', 'with', 'data', 'it', 'employs', 'techniques', 'and', 'theories', 'drawn', 'from', 'many', 'fields', 'within', 'the', 'context', 'of', 'mathematics', 'statistics', 'information', 'science', 'and', 'computer', 'science'], tags=[0])

In [19]:
test_answer

['data',
 'science',
 'is',
 'the',
 'field',
 'of',
 'study',
 'that',
 'combines',
 'domain',
 'expertise',
 'programming',
 'skills',
 'and',
 'knowledge',
 'of',
 'math',
 'and',
 'statistics',
 'to',
 'extract',
 'meaningful',
 'insights',
 'from',
 'data',
 'data',
 'science',
 'practitioners',
 'apply',
 'machine',
 'learning',
 'algorithms',
 'to',
 'numbers',
 'text',
 'images',
 'video',
 'audio',
 'and',
 'more',
 'to',
 'produce',
 'artificial',
 'intelligence',
 'ai',
 'systems',
 'that',
 'perform',
 'tasks',
 'which',
 'ordinarily',
 'require',
 'human',
 'intelligence',
 'in',
 'turn',
 'these',
 'systems',
 'generate',
 'insights',
 'that',
 'analysts',
 'and',
 'business',
 'users',
 'translate',
 'into',
 'tangible',
 'business',
 'value']

In [6]:
model.build_vocab([train_answer])

In [7]:
%time model.train([train_answer], total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 258 ms, sys: 47.2 ms, total: 305 ms
Wall time: 268 ms


In [8]:
inferred_vector = model.infer_vector(train_answer.words)

In [9]:
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

In [10]:
sims

[(0, 0.9573736786842346)]

In [11]:
inferred_vector1 = model.infer_vector(test_answer)

In [12]:
sims1 = model.docvecs.most_similar([inferred_vector1], topn=len(model.docvecs))

In [13]:
sims1

[(0, 0.8895453214645386)]

In [14]:
p_ans = perfect_answer.lower().split()
r_ans = recorded_answer.lower().split()
stops = set(stopwords.words("english"))
p_ans = [w for w in p_ans if not w in stops]
r_ans = [w for w in r_ans if not w in stops]
p_ans = ' '.join(p_ans)
r_ans = ' '.join(r_ans)
p_ans = re.sub(r'[^a-zA-Z]', " ", p_ans)
r_ans = re.sub(r'[^a-zA-Z]', " ", r_ans)
# Shorten words to their stems
stemmer = SnowballStemmer('english')
p_stemmed_words = [stemmer.stem(word) for word in p_ans]
r_stemmed_words = [stemmer.stem(word) for word in r_ans]
p_ans = " ".join(p_stemmed_words)
r_ans = " ".join(r_stemmed_words)
p_ans = re.compile("[ ]{2,}").split(p_ans)
r_ans = re.compile("[ ]{2,}").split(r_ans)
for i in range(len(p_ans)):
    p_ans[i] = p_ans[i].replace(" ", "")
    
for i in range(len(r_ans)):
    r_ans[i] = r_ans[i].replace(" ", "")
p_ans = " ".join(p_ans)
r_ans = " ".join(r_ans)

In [15]:
p_ans

'data science multi disciplinary field uses scientific methods processes algorithms systems extract knowledge insights data various forms structured unstructured similar data mining data science concept unify statistics data analysis machine learning related methods order understand analyze actual phenomena data employs techniques theories drawn many fields within context mathematics statistics information science computer science '

In [16]:
train_answer = gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(p_ans), [0])
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab([train_answer])
%time model.train([train_answer], total_examples=model.corpus_count, epochs=model.epochs)
inferred_vector = model.infer_vector(train_answer.words)
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
inferred_vector1 = model.infer_vector(r_ans.split())
sims1 = model.docvecs.most_similar([inferred_vector1], topn=len(model.docvecs))

CPU times: user 78.7 ms, sys: 23 ms, total: 102 ms
Wall time: 90 ms


In [17]:
sims

[(0, 0.05153155326843262)]

In [18]:
sims1

[(0, 0.003938394133001566)]